# Exposure bracketing using the sequencer feature of USB3 ace devices

In [1]:
import pypylon.pylon as py
import numpy as np

In [2]:
# open the first USB device
info = py.DeviceInfo()
info.SetDeviceClass("BaslerUsb")
cam = py.InstantCamera(py.TlFactory.GetInstance().CreateFirstDevice(info))

In [3]:
# this code only works for ace USB
if not cam.GetDeviceInfo().GetModelName().startswith("acA"):
    print("_This_ sequencer configuration only works to basler ace USB")
    

In [4]:
# open device
cam.Open()

## setup camera.
sequencer mode is not running if the auto-functions are active

In [5]:
cam.UserSetSelector = "Default"
cam.UserSetLoad.Execute()

In [6]:
# mean exposure time 20ms
exp_0 = 2000
exp_1 = 20000
exp_2 = 200000

### activate chunks [ embedded data ]
This allows to read the exposuretime an image has been taken with from the image data

In [7]:
# enable camera chunk mode
cam.ChunkModeActive = True
# enable exposuretime chunk
cam.ChunkSelector = "ExposureTime"
cam.ChunkEnable = True

### sequencer setup

In [8]:
cam.SequencerMode = "Off"
cam.SequencerConfigurationMode = "On"

### setup set 0

In [9]:
cam.SequencerSetSelector = 0

cam.ExposureTime = exp_0

cam.SequencerSetSave.Execute();

In [10]:
### setup set 1

In [11]:
cam.SequencerSetSelector = 1

cam.ExposureTime = exp_1

cam.SequencerSetSave.Execute();

In [12]:
### setup set 2

In [13]:
cam.SequencerSetSelector = 2

cam.ExposureTime = exp_2

# select that we jump to set 0 after this set
# path 1 is the _next_ path / path 0 would be _reset_ path
cam.SequencerPathSelector = 1
cam.SequencerSetNext = 0

cam.SequencerSetSave.Execute();

### enable sequencer

In [14]:
cam.SequencerConfigurationMode = "Off"

# this will set the first sequencer set to set _0_ as side effect
cam.SequencerMode = "On"

### test capture with enabled sequencer mode

In [15]:
# grab 4 sets of 3 images
cam.StartGrabbingMax( 4 * 3)

while cam.IsGrabbing():
    res = cam.RetrieveResult(1000)
    exp_time_chunk = res.ChunkDataNodeMap.ChunkExposureTime.Value
    print("%d\t%6.0f\t%6.2f" %(res.BlockID, exp_time_chunk, np.mean(res.Array)))
    # ... do something with the images
    
    # return to buffer queue
    res.Release()
    
cam.StopGrabbing()

0	  2000	 17.95
1	 20000	 77.26
2	200000	201.53
3	  2000	 17.94
4	 20000	 77.25
5	200000	201.57
6	  2000	 17.96
7	 20000	 77.28
8	200000	201.56
9	  2000	 17.96
10	 20000	 77.29
11	200000	201.55


### Shutdown session after acquisition

This is required to rerun the notebook

In [16]:
cam.SequencerMode = "Off"
cam.Close()